In [ ]:
# Import required packages
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load pre-trained model
net = cv2.dnn.readNetFromDarknet('yolov2.cfg', 'coins.weights')

In [ ]:
# Read in an store class names
classes = []
with open('classes.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

In [ ]:
# Read in image for detection
path = 'myimage.png'
myimg = cv2.imread(path)
plt.imshow(myimg)

In [ ]:
# Store height and width of the image
ht, wt, _ = myimg.shape

In [ ]:
# Create a blob image using OpenCV
blob = cv2.dnn.blobFromImage(myimg, 1/255, (416, 416), (0, 0, 0), swapRB = True, crop = False)

In [ ]:
# Set blob image as input to the model
net.setInput(blob)

In [ ]:
# Perform object detection
last_layer = net.getUnconnectedOutLayersNames()
layer_out = net.forward(last_layer)

In [ ]:
# Calculate all data for the boxes
boxes = []
confidences = []
class_ids = []
for output in layer_out:
    for detection in output:
        score = detection[5:]
        class_id = np.argmax(score)
        confidence = score[class_id]
        if confidence > .6:
            center_x = int(detection[0] * wt)
            center_y = int(detection[1] * ht)
            w = int(detection[2] * wt)
            h = int(detection[3] * ht)
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
indexes = cv2.dnn.NMSBoxes(boxes, confidences, .5, .4)
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size = (len(boxes), 3))

In [ ]:
# Draw the boxes, insert the text and display the result
if len(indexes) > 0:
    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        conficence = str(round(confidences[i], 2))
        color = colors[i]
        cv2.rectangle(myimg, (x, y), (x + w, y + h), color , 2)
        cv2.putText(myimg, label + " (" + str(confidence) + ")", (x, y+20), font, 2, (0, 0, 0), 2)

    cv2.imshow(path, myimg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No objects were detected.")